In [ ]:
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from os import listdir
from os.path import isfile, join
import os
import matplotlib
matplotlib.use('Agg')

In [ ]:
def extract_name(file):
  c = 0
  for i in file:
    if(i.isalpha()):
      c+=1
    else:
      break
  return file[:c]

In [ ]:
def create_image(dir_path,label,path):
    try:
        mat = scipy.io.loadmat(dir_path+"/"+path,appendmat=False)
    except:
        print("Exception occured")
        return

    if("data" not in mat.keys() and "fs" not in mat.keys() ):
        print("image could not be generated")
        print("Cols are ",mat.keys())
        return

    mat_data = mat['data']
    fs = mat['fs']

    num_samples, num_channels = mat_data.shape

    size = int(num_samples/50)

    for i in range(0,num_samples,size):
      data=mat_data[i:i+size]

    # Create a figure
      fig = plt.figure(frameon = False,clear=True)
      fig.set_size_inches(12,8)
      ax1 = plt.Axes(fig,[0.,0.,1.,1.])
      ax1.set_axis_off()
      fig.add_axes(ax1)

      data_for_specgram = data[:, 0]
      spec, freqs, t, im = ax1.specgram(data_for_specgram, Fs=float(fs), NFFT=256, cmap='magma')


      # Save the image
      idx = int(i/size)
      plt.savefig("./imgs/"+label+"/"+path[:-4]+"_"+str(idx)+"_img.png")
    
      plt.close(fig)

In [ ]:
def create_folder(name):
  path = "./imgs/"+name
  try:
    os.mkdir(path)
  except:
    print("Error in creating path for "+name)

In [ ]:
path = "./HUST bearing a practical dataset for ball bearing fault diagnosis/HUST bearing dataset"

files = [f for f in listdir(path) if isfile(join(path, f))]

s = []

print(len(files))

for file in files:
    label = extract_name(file)
    if(label not in s):
      print(label)
      s.append(label)
      create_folder(label)
    create_image(path,label,file)

In [ ]:
from keras.preprocessing import image
import os
import tensorflow as tf
import pandas as pd

In [ ]:
def load_images_from_path(path, label):
    images = []
    labels = []

    for file in os.listdir(path):
        images.append(image.img_to_array(image.load_img(os.path.join(path, file), target_size=(224, 224, 3))))
        labels.append((label))

    return images, labels

In [ ]:
from os import listdir

x = []
y = []
DRIVE_PATH = "./imgs_50/"
class_labels = listdir(DRIVE_PATH)
print(class_labels)

for i in class_labels:
  with tf.device("CPU"):
    images, labels = load_images_from_path(DRIVE_PATH+i, i)
    x += images
    y += labels


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

y = le.fit_transform(y)

In [ ]:
rom tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.3, random_state=0)

x_train_norm = np.array(x_train) / 255
x_test_norm = np.array(x_test) / 255

y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense 
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(7, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

hist = model.fit(x_train_norm, y_train_encoded, validation_data=(x_test_norm, y_test_encoded), batch_size=10, epochs=10) 

import matplotlib.pyplot as plt
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, '-', label='Training Accuracy')
plt.plot(epochs, val_acc, ':', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.plot()
plt.show()

print(hist.history['accuracy'][-1]*100)

In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet import preprocess_input

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x_train_norm = preprocess_input(np.array(x_train))
x_test_norm = preprocess_input(np.array(x_test))

train_features = base_model.predict(x_train_norm)
test_features = base_model.predict(x_test_norm)

model1 = Sequential()
model1.add(Flatten(input_shape=train_features.shape[1:]))
model1.add(Dense(1024, activation='relu'))
model1.add(Dense(7, activation='softmax'))
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

hist1 = model1.fit(train_features, y_train_encoded, validation_data=(test_features, y_test_encoded), batch_size=10, epochs=10)

acc1 = hist1.history['accuracy']
val_acc1 = hist1.history['val_accuracy']
epochs1 = range(1, len(acc1) + 1)
print(acc1)

plt.plot(epochs1, acc1, '-', label='Training Accuracy')
plt.plot(epochs1, val_acc1, ':', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.plot()
plt.show()

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

## Loading VGG16 model
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False ## Not trainable weights

x_train_norm = preprocess_input(np.array(x_train))
x_test_norm = preprocess_input(np.array(x_test))

train_features = base_model.predict(x_train_norm)
test_features = base_model.predict(x_test_norm)

model2 = Sequential()
model2.add(Flatten(input_shape=train_features.shape[1:]))
model2.add(Dense(1024, activation='relu'))
model2.add(Dense(7, activation='softmax'))
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

hist2 = model2.fit(train_features, y_train_encoded, validation_data=(test_features, y_test_encoded), batch_size=10, epochs=10)


acc2= hist2.history['accuracy']
val_acc2 = hist2.history['val_accuracy']
epochs2 = range(1, len(acc2) + 1)

plt.plot(epochs2, acc2, '-', label='Training Accuracy')
plt.plot(epochs2, val_acc2, ':', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.plot()
plt.show()

In [ ]:
acc = acc[-1]*100
acc1 = acc1[-1]*100
acc2 = acc2[-1]*100

plt.bar(["CNN","MobileNet","VGGNet"],[acc,acc1,acc2])
plt.show()